In [1]:
%load_ext Cython

In [2]:
#%%cython
# miulti 出力を目指す 1/15
import numpy as np
import math

data = np.loadtxt("PATH",       # 読み込みたいファイルのパスを"PATH"と置き換える．
                  delimiter=" ",    # ファイルの区切り文字
                  skiprows=0,       # 先頭の何行を無視するか（指定した行数までは読み込まない）
                  usecols=(0,1,2,3,4,5,6,7)) # 読み込みたい列番号
#data = data[:1000,:]

l = int((data.shape[1])/2)
ll = int(data.shape[0])

######################################################

def distance():
    a = np.zeros((ll, l, l))
    #a = np.zeros(int(len(x)/2),int(len(x)/2)) ではない！！！！！！！！！
    for h in range (ll):
        for i in range(l):
            for j in range(l):
                a[h][i][j] = math.sqrt(((data[h][2*i] - data[h][2*j])**2)+(data[h][2*i+1] - data[h][2*j+1])**2) 
    return a


def velocity():
    dt = 1/6

    vdata = np.zeros([ll-1, l*2])

    for u in range(ll-1):
        vdata[u][:] = (data[u+1][:] - data[u][:])/dt
        
    return vdata


def normalized_direction(vdata):
    normal_dir = np.zeros([ll-1, l*2])
    for p in range(ll-1):
        for q in range(l):
            normal_dir[p][2*q] =  vdata[p][2*q] / (np.sqrt(vdata[p][2*q]**2 + vdata[p][2*q+1]**2)+1e-15) #+1e-15足して0で割るのを防ぐ
            normal_dir[p][2*q+1] = vdata[p][2*q+1] / (np.sqrt(vdata[p][2*q]**2 + vdata[p][2*q+1]**2)+1e-15)
    return normal_dir

def distance_vector():
    dis = np.zeros((ll, l, l*2))
    #a = np.zeros(int(len(x)/2),int(len(x)/2)) ではない！！！！！！！！！
    for h in range (ll):
        for i in range(l):
            for j in range(l):
                dis[h][i][2*j] = data[h][2*i] - data[h][2*j]
                dis[h][i][2*j+1] = data[h][2*i+1] - data[h][2*j+1]

    return dis



def inner1(m, n):
    inn1 = np.zeros((ll-1, l, l))
    #a = np.zeros(int(len(x)/2),int(len(x)/2)) ではない！！！！！！！！！
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                inn1[h][i][j] = m[h][2*i] * n[h][2*j] + m[h][2*i+1] * n[h][2*j+1]
    #inn1 = np.tile(inn1, (1, 2))
    #inn1 = inn1.reshape([ll-1,l,l]) 
    #print('inn1', inn1)
    return inn1

def inner2(m, n):
    inn2 = np.zeros((ll-1, l, l))
    h = i = j = k = 0
    #print(m.shape)
    #n = np.transpose(n, (0,2,1)) 
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                    inn2[h][i][j] = m[h][i][2*j] * n[h][i][2*j] + m[h][i][2*j+1] * n[h][i][2*j+1]
    return inn2

def inner3(m, n):
    inn3 = np.zeros((ll-1, l, int(l/2)))
    h = i = j = k = 0
    for h in range (ll-1):
        for i in range(l):
            for j in range(int(l/2)):
                for k in range(int(l/2)): 
                    inn3[h][i][j] = m[h][i][2*j] * n[h][i][2*k] + m[h][i][2*j+1] * n[h][i][2*k+1]
    inn3 = inn3.reshape(ll-1, l)
    return inn3

def potential():
    mul = np.zeros((ll-1, l, l))
    g = np.zeros((ll-1, l, l))
    ep = 0.7 # 独立度的な 
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                if (ad[h][i][j] == 0):
                    g[h][i][j] = 0
                else:
                    g =  1 / (ad[h][i][j]**3 ) * (inner1(normal_dir, normal_dir) - 3 * ep / (ad[h][i][j]**2) * np.multiply(inner2(normal_dird, disd),inner2(normal_dird, disd).transpose(0,2,1)))
    return g

def average(a):
    ave = a/(sum(a))
    return ave

def min_max(x, axis=None):
    x_min = x.min(axis=axis, keepdims=True)
    x_max = x.max(axis=axis, keepdims=True)
    return (x - x_min) / (x_max - x_min)


def do_tpm(z3):
    tpm = np.zeros((2**l, l))
    z4 = np.zeros([ll-1,l])

    for p in range(ll-1):
        for q in range(l):
           z4[p][q] = z3[p][l-q-1]#列ベクトルを逆順にする
    
    temp = np.zeros((2**l, l)) + 1
    tp = np.zeros((2**l)) + 1
    tpm = np.zeros((2**l, l)) 
    tpp = np.zeros((2**l)) #各アドレスは
    for i in range(ll-2):
        for j in range(2**l):
            bin_str = format(j, '04b') #lによって変えないといけない
            for k in range(l):
                if int(bin_str[k]): #文字列の右からk番目が1のとき
                    temp[j][:] *= z4[i][k] #時系列データの表す確率をかける
                elif z4[i][k]>=0:
                    temp[j][:] *= (1-z4[i][k])/((ll-1)*l)
                elif z4[i][k]<0:
                    temp[j][:] *= -(1+z4[i][k])/((ll-1)*l)
                tp[j]*=temp[j][k] #各状態の現れる確率を計算しておくtp[j]=temp[j][:] #各状態の現れる確率をk計算しておく
                temp[j][k] *= z4[i+1][k]
            tpm[j][:] += temp[j][:]
            tpp[j] += tp[j] #引数指定しないと変なふうにたされる。要確認
            temp = np.zeros((2**l, l)) + 1
            tp = np.zeros((2**l)) + 1
    return tpp, tpm #averageとるとconditional independenceでなくるるけど，when not average, fai got 0.

a = distance()
ad = np.delete(a, ll-1, 0) #shape(ll-1, l, l)

dis = distance_vector() 
disd = np.delete(dis, ll-1, 0) #shape(ll-1, l, l*2)

vdata = velocity()
normal_dir = normalized_direction(vdata) #shape(ll-1, l*2)
normal_dird = np.tile(normal_dir.reshape(ll-1,1,l*2),(1,l,1))
#normal_dir = np.tile(normal_dir , (1, 2))
#normal_dird = normal_dir.reshape([ll-1,l,2*l])

g = potential()
z3 = np.sum(g, axis=2)
z3 = average(z3)
#print(z3)
tpp, tpm = do_tpm(z3)
ave_tpm = min_max(tpm)
ave_tpp = average(tpp)
print(ave_tpm)
print(ave_tpp)

OSError: PATH not found.

In [ ]:
!pip install pyphi
import pyphi
pyphi.conf.PyphiConfig.CUT_ONE_APPROXIMATION = True
labels = ('A','B','C','D')
l=4
ll=5400
tpm = ave_tpm
tpp = ave_tpp
c = np.zeros((2**l))
innter_list = []
for i in range(2**l):
    innter_list.append(tuple(map(int, (format(i, '04b'))))) #lの値によって変える必要あり
for t in range(2**l):
    state = innter_list[2**l-t-1]
    network = pyphi.Network(tpm, node_labels = labels)
    subsystem = pyphi.Subsystem(network, state)
    c[t] = pyphi.compute.phi(subsystem)
print(c)
bigphi = np.dot(c, tpp)
print(bigphi)